# Imports
---
Import new skis / snowboards to an SQL database

In [452]:
import json
import pymysql
import re

In [453]:
# Database Setup
# --------------------------------------------------
def setupdb():
    f = open('../application/config/localdb_config.json')
    dbconfig = json.loads(f.read())
    db = pymysql.connect(host=dbconfig['localhost'], user=dbconfig['username'], password=dbconfig['password'], database=dbconfig['database'])
    f.close()
    return db

In [454]:
db = setupdb()
cursor = db.cursor()

In [455]:
params = {
    'size': ['size', 'length'],
    'nose width': ['nose width', 'tip width'],
    'waist width': ['waist width'],
    'tail width': ['tail width'],
    'sidecut': ['sidecut radius', 'turning radius', 'radius', 'sidecut'],
    'setback': ['stance setback', 'setback'],
    'effective edge': ['effective edge', 'running length']
}

In [456]:
values = {
    'size': '',
    'nose width': '',
    'waist width': '',
    'tail width': '',
    'sidecut': '',
    'setback': '',
    'effective edge': ''
}

In [457]:
# Extraction of data 
# from string format to a tabular structure
# --------------------------------------------------
def extract_data(table):
    # Remove units from headings
    while ')' in table:
        table = table[:table.find('(')] + table[table.find(')') + 1:]
        
    breakpoints = []
    
    # Find location of headings in table
    for i in params:
        row = []
        for j in params[i]:
            if j in table:
                breakpoints.append(table.find(j))
                break
        
        values[i] = row
        
    # Extract table data into rows
    rows = []
    breakpoints.sort()
    for i, b in enumerate(breakpoints):
        if i > 0:
            rows.append(table[breakpoints[i-1]:b])
        if i == len(breakpoints) -1:
            rows.append(table[b:])
    
    
    # Remove lables from rows
    for row in rows:
        for i in params:
            for j in params[i]:
                if j in row:
                    values[i] = row.replace(j, '')
                    break
                    
    
    # Format table rows as lists
    for val in values:
        try:
            values[val] = values[val].strip()
            values[val] = values[val].split()   
        except Exception as e:
            x = e
            
    
    # Truncate long rows to the correct size
    num_sizes = len(values['size'])
    for val in values:
        values[val] = values[val][:num_sizes]
        
    return values

In [458]:
# Save skiboard in MySQL DB
# --------------------------------------------------
def save_skiboard(skiboard):
    try:
        sql = f"""INSERT INTO skiboards (
            url,
            brand,
            model,
            year,
            name,
            slug,
            category,
            family,
            description,
            stiffness,
            flex_profile,
            camber_profile,
            camber_details,
            core,
            laminates,
            base,
            weight,
            youth
        ) VALUES (
            '{skiboard['url']}',
            '{skiboard['brand']}',
            '{skiboard['model']}',
            '{skiboard['year']}',
            '{str(skiboard['brand']).capitalize()} {str(skiboard['model']).capitalize()} {str(skiboard['year'])}',
            '{str(skiboard['brand']).lower()}-{str(skiboard['model']).lower()}-{str(skiboard['year'])}',
            '{skiboard['category']}',
            '{skiboard['family']}',
            '{skiboard['description']}',
            '{skiboard['stiffness']}',
            '{skiboard['flex_profile']}',
            '{skiboard['camber_profile']}',
            '{skiboard['camber_details']}',
            '{skiboard['core']}',
            '{skiboard['laminates']}',
            '{skiboard['base']}',
            '{skiboard['weight']}',
            '{skiboard['youth']}'
        )"""
        cursor.execute(sql)
        db.commit()
    except Exception as e:
        print(f"ERROR:\n{e}")

In [480]:
# Save data in MySQL DB
# --------------------------------------------------
def save_values(skiboard, values):
    print(f"Importing...\n{str(skiboard['brand']).capitalize()} {str(skiboard['model']).capitalize()} {str(skiboard['year'])}\nValues:\n{values}")

    for x in range(len(values['size'])):
        try:
            sql = f"""REPLACE INTO sizes (
                skiboard_id,
                size,
                nose_width,
                waist_width,
                tail_width,
                sidecut,
                setback,
                effective_edge
            ) VALUES (
                '{skiboard['id']}',
                '{values['size'][x]}',
                '{values['nose width'][x]}',
                '{values['waist width'][x]}',
                '{values['tail width'][x]}',
                '{values['sidecut'][x]}',
                '{values['setback'][x]}',
                '{values['effective edge'][x]}'
            )"""

            cursor.execute(sql)
            db.commit()
        except Exception as e:
            print(f"ERROR: size ({values['size'][x]})\n{e}\n")
    
    return True

# Create New SkiBoard
---

In [481]:
skiboard = {
    'id': 21,
    'url': '',
    'brand': 'Burton',
    'model': 'Leader Board',
    'year': 2020,
    'category': 'Snowboard',
    'family': 'Family Tree',
    'description': "Made to excel on the Freeride World Tour, the Burton Family Tree Leader Board Snowboard brings confidence and stability to sketchy situations. From the slightly spooned nose to minimize edge catch in no-fall zones, to the aggressive flex that'll hold on when the snow doesn't, to a perfect taper that keeps the nose up without turning this board into a whimsical wiggler: the Leader Board is the one you want under your feet when the going gets marginal. This one's high strung and responsive because it has to be.".replace("'", "`"),
    'stiffness': 8,
    'flex_profile': 'Directional',
    'camber_profile': 'Directional Camber',
    'camber_details': '',
    'core': 'Super Fly II™ 700G Core',
    'laminates': 'Carbon I-Beam',
    'base': 'Recycled Sintered WFO',
    'weight': 0,
    'youth': 0
}

In [482]:
save_skiboard(skiboard)

# Add Sizes to SkiBoard
---

In [483]:
table = input("Please submit your ski/snowboard size table\n\n").lower()

Please submit your ski/snowboard size table

Size (cm)	158	162	166 Effective Edge (mm)	1215	1255	1260 Tip Width (mm)	295	299.4	305.7 Waist Width (mm)	252	255	260 Tail Width (mm)	285	289.4	295.7 Sidecut Radius (m)	8.6	8.8	9.2 Stance Setback (mm)	25	25	25 Rider Weight (lbs)	150 - 200	180 - 260​+	180 - 260​+ Width	Regular	Regular	Regular


In [484]:
values = extract_data(table)

In [485]:
for row in values:
    print(f"{row}:\n{values[row]}\n\n")

size:
['158', '162', '166']


nose width:
['295', '299.4', '305.7']


waist width:
['252', '255', '260']


tail width:
['285', '289.4', '295.7']


sidecut:
['8.6', '8.8', '9.2']


setback:
['25', '25', '25']


effective edge:
['1215', '1255', '1260']




In [486]:
success = save_values(skiboard, values)
if success:
    print("SUCCESS!")

Importing...
Burton Leader board 2020
Values:
{'size': ['158', '162', '166'], 'nose width': ['295', '299.4', '305.7'], 'waist width': ['252', '255', '260'], 'tail width': ['285', '289.4', '295.7'], 'sidecut': ['8.6', '8.8', '9.2'], 'setback': ['25', '25', '25'], 'effective edge': ['1215', '1255', '1260']}
SUCCESS!
